### Practice using Pytorch with MNIST dataset
### practice using covnet


In [23]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

In [24]:
# check if we have cuda enabled
torch.cuda.is_available()

True

In [25]:
print(os.listdir('.'))

['data', '.idea', '.gitignore', 'input', 'logistic_practice.ipynb', '.git', 'prac.ipynb', 'notebook.ipynb']


In [26]:
# prepare datasets
# load data
train = pd.read_csv('data/train.csv', dtype=np.float32)

# split data into pixels and labels  (our features are the pixels)
targets_numpy = train['label'].values

# features will be all categories except our target, and then standardize all values
features_numpy = train.loc[:, train.columns != 'label'].values / 255

X_train, X_test, y_train, y_test = train_test_split(features_numpy, targets_numpy, test_size=0.2, random_state=42)

In [27]:
X_train_tensor = torch.from_numpy(X_train)
# convert target to type long
y_train_tensor = torch.from_numpy(y_train).type(torch.LongTensor)

X_test_tensor = torch.from_numpy(X_test)
y_test_tensor = torch.from_numpy(y_test).type(torch.LongTensor)

In [36]:


# Create CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1
        self.fc1 = nn.Linear(32 * 4 * 4, 10)

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2
        out = self.maxpool2(out)

        # flatten
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)

        return out

In [37]:
batch_size = 100
n_iters = 2500
epochs = int(n_iters / (len(X_train) / batch_size))
epochs

7

In [38]:
# pytorch train and test sets
train = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
test = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

# dataloader

train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test, batch_size, shuffle=False)

In [40]:
# initialize model, error, and optimizer
model = CNNModel()

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [42]:
# Train model
loss_list = []
iterations = []
accuracy = []
count = 0
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)
        #clear gradients
        optimizer.zero_grad()
        # fp
        outputs = model(train)
        # loss function
        loss = criterion(outputs, labels)
        # bp
        loss.backward()
        # update
        optimizer.step()
        count += 1

        if count % 50 == 0:
            correct = 0
            total = 0
            # iterate through test set
            for images, labels in test_loader:
                test = Variable(images.view(100, 1, 28, 28))

                outputs = model(test)

                predicted = torch.max(outputs.data, 1)[1]

                total += len(labels)

                correct += (predicted == labels).sum()

            acc = 100 * correct / float(total)

            loss_list.append(loss.data)
            iterations.append(epoch)
            accuracy.append(acc)
        if count % 500 == 0:
            print('iteration {} loss {} accuracy {}'.format(count, loss.data, acc))


iteration 500 loss 0.09391464293003082 accuracy 96.94047546386719
iteration 1000 loss 0.03574056550860405 accuracy 98.0
iteration 1500 loss 0.025812659412622452 accuracy 98.0
iteration 2000 loss 0.012040985748171806 accuracy 98.36904907226562
